## Mortgage Loans: Logistic Regression Example

In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt

In [12]:
df = pd.read_csv('../data/loan_data_set.csv')
df.shape

(614, 13)

In [13]:
features = ['Credit_History','LoanAmount','Loan_Amount_Term','ApplicantIncome']

In [14]:
for feature in features:
    df[feature].fillna(value=df[feature].mean(), inplace=True)
print(df.shape)
df.isnull().sum()

(614, 13)


Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64

In [15]:
df['Loan_Approval']=np.where(df['Loan_Status']=="Y", 1, 0)

In [16]:
y = df['Loan_Approval']
X = df[features]

In [17]:
X.dtypes

Credit_History      float64
LoanAmount          float64
Loan_Amount_Term    float64
ApplicantIncome       int64
dtype: object

In [18]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=12)

In [19]:
# Fit the model to the training dataset
mymodel = LogisticRegression()
mymodel.fit(X_train, y_train)

LogisticRegression()

In [20]:
# coefficients and intercept
mymodel.intercept_
mymodel.coef_

array([[ 3.05839266e+00, -2.55748576e-03, -5.22538765e-03,
         2.21930464e-05]])

In [21]:
# what is the equation for this mymodel?
cols=list(X.columns)
coefs=list(mymodel.coef_)
xcoefs = [(coefs[i], cols[i]) for i in range(0, len(cols))]
output = f'y = {round(mymodel.intercept_,2)} + {round(xcoefs[0][0],2)}*{xcoefs[0][1]} + {round(xcoefs[1][0],2)}*{xcoefs[1][1]} + {round(xcoefs[2][0],2)}*{xcoefs[2][1]} + {round(xcoefs[3][0],2)}*{xcoefs[3][1]} '
print("Regression Equation: ", output)  

IndexError: list index out of range

In [6]:
# Predict the y-values on the testing dataset
y_preds = mymodel.predict(X_test)

NameError: name 'mymodel' is not defined

In [7]:
# root mean sq error
rmse = sqrt(metrics.mean_squared_error(y_test, y_preds))
rmse

NameError: name 'y_test' is not defined

In [ ]:
metrics.classification_report(y_true, y_pred)